In [3]:
import os
import json
import numpy as np
from langchain.llms import HuggingFaceHub
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.callbacks.tracers import LangChainTracer
from langchain.callbacks.manager import CallbackManager

from transformers import AutoTokenizer

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your-hugging-token-face-token-api"  # Use your token
os.environ["LANGCHAIN_API_KEY"] = "your-langsmith-token-api"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [5]:
# Load FAQ dataset
with open("Ecommerce_FAQ_Chatbot_dataset.json", "r") as f:
    data = json.load(f)

documents = []
for entry in data["questions"]:
    q = entry["question"].strip()
    a = entry["answer"].strip()
    documents.append(f"Q: {q}\nA: {a}")

In [6]:
print("\nProcessed Document List (first 3 docs shown):")
for d in documents[:3]:
    print(d)


Processed Document List (first 3 docs shown):
Q: How can I create an account?
A: To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.
Q: What payment methods do you accept?
A: We accept major credit cards, debit cards, and PayPal as payment methods for online orders.
Q: How can I track my order?
A: You can track your order by logging into your account and navigating to the 'Order History' section. There, you will find the tracking information for your shipment.


In [7]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipykernel_103860/412152783.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [8]:
# Tokenize documents for debugging output
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
tokens = tokenizer(documents, padding=True, truncation=True, return_tensors='pt')
print("\nTokenization Output for Documents (showing input_ids and attention_mask for first doc):")
print("Input IDs:", tokens["input_ids"][0])
print("Attention Mask:", tokens["attention_mask"][0])


Tokenization Output for Documents (showing input_ids and attention_mask for first doc):
Input IDs: tensor([  101,  1053,  1024,  2129,  2064,  1045,  3443,  2019,  4070,  1029,
         1037,  1024,  2000,  3443,  2019,  4070,  1010, 11562,  2006,  1996,
         1005,  3696,  2039,  1005,  6462,  2006,  1996,  2327,  2157,  3420,
         1997,  2256,  4037,  1998,  3582,  1996,  8128,  2000,  3143,  1996,
         8819,  2832,  1012,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0])
Attention Mask: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [9]:
# Generate embeddings for documents
raw_embeddings = embedding_model.embed_documents(documents)
print("\nEmbeddings Output:")
for i, emb in enumerate(raw_embeddings[:3]):
    print(f"Document {i+1} embedding shape: {len(emb)}")



Embeddings Output:
Document 1 embedding shape: 384
Document 2 embedding shape: 384
Document 3 embedding shape: 384


In [10]:
# Normalize embeddings to ensure cosine similarity behavior.
def normalize(emb):
    arr = np.array(emb)
    norm = np.linalg.norm(arr)
    return (arr / norm).tolist() if norm != 0 else emb

normalized_embeddings = [normalize(emb) for emb in raw_embeddings]


In [11]:
# Combine documents with their normalized embeddings as (document, embedding) pairs.
doc_embeddings = list(zip(documents, normalized_embeddings))


In [12]:
vectorstore = FAISS.from_embeddings(doc_embeddings, embedding_model)

In [13]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [14]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")

/tmp/ipykernel_103860/3634644841.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")


In [15]:
tracer = LangChainTracer(project_name="agent-chatbot")
callback_manager = CallbackManager([tracer])

In [16]:
def distance_to_similarity(distance):
    return 1 / (1 + distance)


In [17]:
sample_query = "How can I create an account?"
results = vectorstore.similarity_search_with_score(sample_query, k=3)
print("\nSimilarity search results (with converted similarity scores):")
for i, (doc, score) in enumerate(results, start=1):
    similarity = distance_to_similarity(score)
    print(f"\n[Document {i}]:\n{doc.page_content}")
    print(f"Distance Score: {score:.12f}, Converted Similarity: {similarity:.12f}")


Similarity search results (with converted similarity scores):

[Document 1]:
Q: How can I create an account?
A: To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.
Distance Score: 0.293128192425, Converted Similarity: 0.773318529129

[Document 2]:
Q: Can I order without creating an account?
A: Yes, you can place an order as a guest without creating an account. However, creating an account offers benefits such as order tracking and easier future purchases.
Distance Score: 1.007702350616, Converted Similarity: 0.498081803322

[Document 3]:
Q: Do you have a loyalty program?
A: Yes, we have a loyalty program where you can earn points for every purchase. These points can be redeemed for discounts on future orders. Please visit our website to learn more and join the program.
Distance Score: 1.459196686745, Converted Similarity: 0.406636863947


In [18]:
# Initialize the LLM 
llm = HuggingFaceHub(
    repo_id="google/flan-t5-small",
    model_kwargs={"max_new_tokens": 150, "temperature": 0.7},
    callback_manager=callback_manager
)

/tmp/ipykernel_103860/2412962749.py:2: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(
/tmp/ipykernel_103860/2412962749.py:2: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = HuggingFaceHub(


In [19]:
# Build the Conversational Retrieval Chain using the LLM and retriever.
faq_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    callback_manager=callback_manager,
    return_source_documents=True,
    verbose=True
)

In [23]:
# Chatbot loop
print("FAQ Chatbot is ready. Type your questions (or type 'exit' to quit):")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting the chatbot session.")
        break

    response = faq_chain({"question": user_input})
    print("Bot:", response["answer"])
    
    # Retrieve top-3 documents with scores using the query,
    # showing converted similarity for display.
    results = vectorstore.similarity_search_with_score(user_input, k=3)
    for i, (doc, score) in enumerate(results, start=1):
        similarity = distance_to_similarity(score)
        print(f"\n[Source Document {i}]:\n{doc.page_content}")
        print(f"Distance Score: {score:.12f}, Converted Similarity: {similarity:.12f}")

FAQ Chatbot is ready. Type your questions (or type 'exit' to quit):


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Is there a mobile app available for placing orders and tracking shipments?
Assistant: Yes, there is a mobile app available for placing orders and tracking shipments.
Human: quit 
Assistant: Helpful
Human: What’s the best way to reach your customer service if I have an issue with my order?
Assistant: Customer service is a priority.
Follow Up Input: Can I update or change my payment method after placing an order?
Standalone question:


/home/astechware/Projects/NLP&LLM basics/.conda/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Q: Can I change or cancel an item in my order?
A: If you need to change or cancel an item in your order, please contact our customer support team as soon as possible. We will assist you with the necessary steps.

Q: Can I change my shipping address after placing an order?
A: If you need to change your shipping address, please contact our customer support team as soon as possible. We will do our best to update the address if the order has not been shipped yet.

Q: What payment methods do you accept?
A: We accept major credit cards, debit cards, and PayPal as payment methods for online orders.

Question: Is there a way to change my payment method after placing an order?
Helpful Answer:


/home/astechware/Projects/NLP&LLM basics/.conda/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



> Finished chain.

> Finished chain.
Bot: We accept major credit cards, debit cards, and PayPal as payment methods for online orders.

[Source Document 1]:
Q: Can I change or cancel an item in my order?
A: If you need to change or cancel an item in your order, please contact our customer support team as soon as possible. We will assist you with the necessary steps.
Distance Score: 0.856783986092, Converted Similarity: 0.538565635681

[Source Document 2]:
Q: Can I change my shipping address after placing an order?
A: If you need to change your shipping address, please contact our customer support team as soon as possible. We will do our best to update the address if the order has not been shipped yet.
Distance Score: 0.944577336311, Converted Similarity: 0.514250576496

[Source Document 3]:
Q: What payment methods do you accept?
A: We accept major credit cards, debit cards, and PayPal as payment methods for online orders.
Distance Score: 1.167297601700, Converted Similarity: 0.46140411